In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp

In [2]:
with open('all_permutations.pickle', 'rb') as fil:
    all_permutations = pickle.load(fil) 

In [3]:
keys = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'F12', 'F13', 'F14', 'F15', 'F16','F23', 'F24', 'F25', 'F26', 'F34', 'F35', 'F36', 'F45', 'F46', 'F56']

In [4]:
all_perms = [from_labels_to_perm(labels) for labels in all_permutations]

In [5]:
Gr = PermutationGroup(all_perms)

In [6]:
order3_perm = []
for perm in all_perms:
    if perm.order() == 3:
        order3_perm.append(perm)

In [7]:
len(order3_perm)

800

In [11]:
labels_order3 = [from_perm_to_labels(perm) for perm in order3_perm]

In [76]:
index = 0

In [77]:
order3_perm[index]

(3,27,4)(5,26,24)(6,25,23)(7,18,19)(8,14,15)(9,10,13)

In [73]:
print(labels_order3[index])

['E1', 'E2', 'E5', 'E3', 'E4', 'E6', 'G1', 'G2', 'G5', 'G3', 'G4', 'G6', 'F12', 'F15', 'F13', 'F14', 'F16', 'F25', 'F23', 'F24', 'F26', 'F35', 'F45', 'F56', 'F34', 'F36', 'F46']


In [15]:
L_set_base = ['E1', 'G4', 'E2', 'G3', 'E3', 'E5']

In [16]:
L_set_permuted = [labels_order3[index][keys.index(label)] for label in L_set_base]
L_set_permuted

['E1', 'F12', 'E2', 'G4', 'F56', 'F46']

In [43]:
var('xx')
Q.<i> = NumberField(xx^2+3)

In [44]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)

In [45]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [46]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [47]:
line = Line([y, z])

In [48]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [49]:
base_five_points = get_five_points_in_general_position(general_cubic.get_L_set_in_plucker(L_set_base[0:5]))

In [50]:
mat = find_projectivity(base_five_points, general_cubic.get_L_set_in_plucker(L_set_permuted[0:5]))

In [55]:
cl = general_cubic.cl_lines

In [57]:
#cl['E1'] == cl['E1'].apply_proj(mat)

In [58]:
#cl['G4'] == cl['F12'].apply_proj(mat)

In [59]:
#cl['E2'] == cl['E2'].apply_proj(mat)

In [60]:
#cl['G3'] == cl['G4'].apply_proj(mat)

In [61]:
#cl['E3'] == cl['F56'].apply_proj(mat)

In [31]:
conds1 = list(set(matrix([cl['F12'].plucker, cl['G3'].apply_proj(mat).plucker]).minors(2)))

In [32]:
conds2 = list(set(matrix([cl['F56'].plucker, cl['E4'].apply_proj(mat).plucker]).minors(2)))

In [33]:
conds3 = list(set(matrix([cl['F46'].plucker, cl['F36'].apply_proj(mat).plucker]).minors(2)))

In [34]:
conds4 = list(set(matrix([cl['E5'].plucker, cl['F46'].apply_proj(mat).plucker]).minors(2)))

In [36]:
#gb = P.ideal(list(set(conds1+conds2+conds3+conds4))).groebner_basis() 

In [62]:
vrs = P.gens()[0:4]
mon = (sum(vrs) ^ 3).monomials()
change_coord = vector(vrs) * mat
sost = {vrs[i]: change_coord[i] for i in range(4)}
new_cubic = general_cubic.eqn.subs(sost)

In [63]:
coeffs = list(set(matrix([[general_cubic.eqn.coefficient(mn) for mn in mon], [new_cubic.coefficient(mn) for mn in mon]]).minors(2)))

In [64]:
conds = [remove_sing_factors(el, general_cubic.sing_cubic) for el in coeffs if el != 0]

In [70]:
gb1 = P.ideal(conds).radical().primary_decomposition()
gb1

[Ideal (d - e + f, c + e, b - 2*f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 3,
 Ideal (d + e - f, c + f, b - 2*e) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 3,
 Ideal (e - f, b + d - 2*f, c*d - c*f - f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 3,
 Ideal (e + f, d, 3*c + (-i)*f, b) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 3,
 Ideal (e + f, d, 3*c + (i)*f, b) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 3,
 Ideal (d - 2*e, c - f, b + e - f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 3,
 Ideal (d - 2*f, c - e, b - e +

In [66]:
gb1[3]

Ideal (e + f, d, 3*c + (-i)*f, b) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 3

In [67]:
nc = general_cubic.subs({e:-f, d:0, b:0}).subs({c:1/3 *i* f})
nc

-6*x^2*y + 6*x*y^2 - 2*x*z^2 + 4*x*z*t - 4*y*z*t + 2*y*t^2

18

In [58]:
cl['F56'] == cl['E4'].apply_proj(mat)

False

In [59]:
cl['F46'] == cl['F36'].apply_proj(mat)

False

In [60]:
cl['E5'] == cl['F46'].apply_proj(mat)

False

In [9]:
all_fixed_lines3 = []
for perm in order3_perm:
    fixed_lines = []
    for tup in perm.cycle_tuples(singletons=True):
        if len(tup) ==1:
            fixed_lines.append(tup[0])
    if fixed_lines !=[]:
        all_fixed_lines3.append(fixed_lines)

In [10]:
all_fixed_lines3[0]

[1, 2, 11, 12, 16, 17, 20, 21, 22]

In [11]:
[keys[el-1] for el in all_fixed_lines3[0]]

['E1', 'E2', 'G5', 'G6', 'F15', 'F16', 'F25', 'F26', 'F34']

In [12]:
len(all_fixed_lines3)

240

In [13]:
order5_perm = []
for perm in all_perms:
    if perm.order() == 5:
        order5_perm.append(perm)

In [14]:
len(order5_perm)

5184

In [15]:
all_fixed_lines5 = []
for perm in order5_perm:
    fixed_lines = []
    for tup in perm.cycle_tuples(singletons=True):
        if len(tup) ==1:
            fixed_lines.append(tup[0])
    if fixed_lines !=[]:
        all_fixed_lines5.append(fixed_lines)

In [16]:
for lines in all_fixed_lines5:
    if len(lines) !=2:
        print(lines)

In [17]:
E6 = WeylGroup(['E', 6]).as_permutation_group()
E6.gens_small()

[(1,12,14,5,24,9,7,16)(2,26,8,10,23,11,15,27)(3,20)(4,25,17,22,19,13,6,18),
 (1,19,12,21,14,13,3,26,4,27)(2,24,10,17,20)(5,22)(6,23,18,8,16)(7,15,25,11,9)]

In [31]:
ord3 = []
for el in E6:
    if el.order() == 3:
        ord3.append(el)

In [32]:
len(ord3)

800

In [33]:
ord5 = []
for el in E6:
    if el.order() == 5:
        ord5.append(el)

In [34]:
len(ord5)

5184